In [ ]:
import json
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import skimage.io as io
import numpy as np

# hough transform stuff

In [ ]:
data = json.load(open('/root/data/small_pen_data_collection/gtsf_images/gtsf_labels.json'))

In [ ]:
img = io.imread('/root/data/small_pen_data_collection/gtsf_images/cjhk4p9dyf5va0702liitudpz%2Ffd8fcf11-e1c6-49d5-8c4e-ad6b21039533%2FIMG_1010002.JPG?alt=media')

In [ ]:
img.shape

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)

In [ ]:
# lines = cv2.HoughLines(edges,1,np.pi/180,200)

# plt.figure(figsize=(20, 10))
# plt.imshow(img)

# for rho,theta in lines.squeeze():
#     a = np.cos(theta)
#     b = np.sin(theta)
#     x0 = a*rho
#     y0 = b*rho
#     x1 = int(x0 + 1000*(-b))
#     y1 = int(y0 + 1000*(a))
#     x2 = int(x0 - 1000*(-b))
#     y2 = int(y0 - 1000*(a))

#     # cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
#     plt.plot([x1, x2], [y1, y2])
    
# plt.axis('off')
# plt.show()

# xy axis

In [ ]:
import pandas as pd

In [ ]:
gtdata = pd.read_csv('/root/data/small_pen_data_collection/gtsf.csv')

In [ ]:
ground_truth_lengths = {}
for (i,exp) in enumerate(gtdata.iloc[4:, 0]):
    ground_truth_lengths[exp] = gtdata.iloc[i+4, 3]

In [ ]:
data = json.load(open('/root/data/small_pen_data_collection/gtsf_images/gtsf_labels.json'))

In [ ]:
x_axis_length = 900
y_axis_length = 600

In [ ]:
data_subset = []
for d in data:
    if 'Y axis' in d['Label'] and 'X axis' in d['Label']:
        data_subset.append(d)

In [ ]:
len(data_subset)

In [ ]:
truths = []
preds = []
for d in data_subset:
    # print(d['Dataset Name'])
    
    exp = d['Dataset Name'].split('-')[0]
#     if exp not in ['181017010010', '181017010011']:
#         continue
    # print(d['External ID'])
    total_l_coordinates = d['Label']['total L (mouth to caudal fin)'][0]['geometry']
    # print(total_l_coordinates)
    total_l_pixel_length_x = np.abs(total_l_coordinates[0]['x'] - total_l_coordinates[1]['x'])
    total_l_pixel_length_y = np.abs(total_l_coordinates[0]['y'] - total_l_coordinates[1]['y'])
    print(total_l_pixel_length_x, total_l_pixel_length_y)
    
    x_axis_coordinates = d['Label']['X axis'][0]['geometry']
    x_axis_pixel_length = np.abs(x_axis_coordinates[0]['x'] - x_axis_coordinates[1]['x'])
    y_axis_coordinates = d['Label']['Y axis'][0]['geometry']
    y_axis_pixel_length = np.abs(y_axis_coordinates[0]['y'] - y_axis_coordinates[1]['y']) 
    print(x_axis_pixel_length, y_axis_pixel_length)
    
    # PYTHAGORE MOTHERFUCKER
    total_length_mm_x = (total_l_pixel_length_x * x_axis_length) / x_axis_pixel_length
    print(total_length_mm_x)
    total_length_mm_y = (total_l_pixel_length_y * y_axis_length) / y_axis_pixel_length
    print(total_length_mm_y)
    total_length_mm = np.sqrt(total_length_mm_x**2 + total_length_mm_y**2)
    print("predicted length {}".format(total_length_mm))
    
    gt_length = int(ground_truth_lengths[exp])
    print("gt length {}".format(gt_length))
    rel_error = (total_length_mm-gt_length) / gt_length
    print("relative error {}".format(rel_error))
    print('#'*100)
    truths.append(gt_length)
    preds.append(total_length_mm)

In [ ]:
plt.scatter(preds, truths)
plt.show()

In [ ]:
data[0]